<a href="https://colab.research.google.com/github/json724/pycon2023/blob/embeddings_vectordb/pycon2023_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update &&  apt install ffmpeg -y
!pip install -U sentence-transformers
!pip install tqdm
!pip install git+https://github.com/openai/whisper.git
!pip install --upgrade --no-cache-dir gdown
!pip install pytube
!pip install pinecone-client
!pip install chromadb

In [ ]:
import gdown
gdrive_folder_link='https://drive.google.com/drive/folders/1f3v_CKRVecCqpdX07PQg5mg19mWRaVJC?usp=sharing'
gdown.download_folder(gdrive_folder_link, quiet=True)

In [4]:
!ls

'PyCon-co audios'   sample_data


In [5]:
import pandas as pd
import os 
import re
from tqdm import tqdm 

In [9]:
title = []
url = []
path = []
patron = r'\[(.*?)\]'
for _ in os.listdir("/content/PyCon-co audios/audios"):
  title.append(_.split('[]')[0])
  url.append(re.findall(patron, _)[0])
  path.append("/content/PyCon-co audios/audios/" + _)


In [11]:
df_video = pd.DataFrame({'title': title, 'url': url, 'path': path})

In [12]:
df_video['url'] = df_video['url'].apply(lambda x: "https://www.youtube.com/watch?v=" + x)

In [14]:
df_video.head()

,title,url,path
0,Jonathan Diaz - Sanic StorageService. Usando S...,https://www.youtube.com/watch?v=d1Dz5oHE6PU,/content/PyCon-co audios/audios/Jonathan Diaz ...
1,José Benitez - The road so far on building a w...,https://www.youtube.com/watch?v=7NlKbzz1y0M,/content/PyCon-co audios/audios/José Benitez -...
2,Jorge Gonzalez - Real driving data to improve ...,https://www.youtube.com/watch?v=E_rPy0h5fXI,/content/PyCon-co audios/audios/Jorge Gonzalez...
3,Interview Flavio Percoco - Elastic - PyCon Col...,https://www.youtube.com/watch?v=wfWzKnILBVs,/content/PyCon-co audios/audios/Interview Flav...
4,Rodolfo Edelmann & Carlos de la Torre - Conect...,https://www.youtube.com/watch?v=N3czkxo2JJw,/content/PyCon-co audios/audios/Rodolfo Edelma...


In [15]:
from pytube import YouTube

In [19]:
def get_metadata(url):
  yt = YouTube(url)
  return pd.Series([yt.views, yt.author, yt.publish_date, yt.keywords])

In [21]:
df_video[['views', 'author', 'publish_date', 'keywords']]= df_video['url'].apply(lambda x: get_metadata(x))

In [22]:
df_video.head()

,title,url,path,views,author,publish_date,keywords
0,Jonathan Diaz - Sanic StorageService. Usando S...,https://www.youtube.com/watch?v=d1Dz5oHE6PU,/content/PyCon-co audios/audios/Jonathan Diaz ...,90,PyCon Colombia,2021-08-15,[]
1,José Benitez - The road so far on building a w...,https://www.youtube.com/watch?v=7NlKbzz1y0M,/content/PyCon-co audios/audios/José Benitez -...,96,PyCon Colombia,2020-03-05,[]
2,Jorge Gonzalez - Real driving data to improve ...,https://www.youtube.com/watch?v=E_rPy0h5fXI,/content/PyCon-co audios/audios/Jorge Gonzalez...,543,PyCon Colombia,2019-07-25,[]
3,Interview Flavio Percoco - Elastic - PyCon Col...,https://www.youtube.com/watch?v=wfWzKnILBVs,/content/PyCon-co audios/audios/Interview Flav...,109,PyCon Colombia,2020-04-04,[]
4,Rodolfo Edelmann & Carlos de la Torre - Conect...,https://www.youtube.com/watch?v=N3czkxo2JJw,/content/PyCon-co audios/audios/Rodolfo Edelma...,503,PyCon Colombia,2020-04-20,[]


In [23]:
df_video.path[4]

'/content/PyCon-co audios/audios/Rodolfo Edelmann & Carlos de la Torre - Conectando microservicios con Python [N3czkxo2JJw].mp3'

In [25]:
import whisper

model = whisper.load_model("base")
result = model.transcribe(df_video.path[4])
print(result["text"])

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.1MiB/s]


 Bueno, quién soy Rodolfo Edelman, me van a escuchar ahí que me dicen Rudy. Tiengan cuidado si me llaman Digan Rudy con Deno con B. Larga, porque con Ferencia Python se pueden ojar a alguien. Hace 5 años estoy trabajando con Python antes de trabajar con Ruby, eso no lo voy a contermas. Trabajé mucho tiempo a la Universidad Nacional de Córdoba, a esa foto en el Cláster de la Universidad Nacional de Córdoba. Hace mucho de Robots, después abancé a Bakken, de Bakken, empezó a hacer Machine Learning y actualmente estoy como técnica al Manaller en Mercado de Libre, desarrollando productos, servicios internos de la compañía que nos sirven para poner modelos de Machine Learning en producción. Hola, yo soy Litox, Carlos de la Torre, me dicen Litox en la Uroschatz y en las comunidades de Python y en la tele, pero eso todavía no se se llenó nunca. Trabajo con Ruy hace muchos años, también soy técnica al Manaller, tengo un recorrido de su licencia informática y de desarrollador, técnica leader, re

In [26]:
result

{'text': ' Bueno, quién soy Rodolfo Edelman, me van a escuchar ahí que me dicen Rudy. Tiengan cuidado si me llaman Digan Rudy con Deno con B. Larga, porque con Ferencia Python se pueden ojar a alguien. Hace 5 años estoy trabajando con Python antes de trabajar con Ruby, eso no lo voy a contermas. Trabajé mucho tiempo a la Universidad Nacional de Córdoba, a esa foto en el Cláster de la Universidad Nacional de Córdoba. Hace mucho de Robots, después abancé a Bakken, de Bakken, empezó a hacer Machine Learning y actualmente estoy como técnica al Manaller en Mercado de Libre, desarrollando productos, servicios internos de la compañía que nos sirven para poner modelos de Machine Learning en producción. Hola, yo soy Litox, Carlos de la Torre, me dicen Litox en la Uroschatz y en las comunidades de Python y en la tele, pero eso todavía no se se llenó nunca. Trabajo con Ruy hace muchos años, también soy técnica al Manaller, tengo un recorrido de su licencia informática y de desarrollador, técnica 

Por tiempo se transcribieron los textos procesando el diccionario interno que contiene, id, seek, start, end, text

In [28]:
df_transcripts = pd.read_csv("/content/PyCon-co audios/all_transcribes.csv", sep="|")

In [30]:
df_transcripts.shape

(5873, 12)

In [31]:
df_transcripts.head()

,title,url,path,views,author,publish_date,keywords,channel_id,position,start,end,text
0,Lorena Aldana - Using Python to listen to your...,https://www.youtube.com/watch?v=YrCSME1jT6U,PyCon-co audios/audios/Lorena Aldana - Using P...,200,PyCon Colombia,2021-09-09,[],UCjor6U0ZF5zGAYLJJt9gr0A,1,0.00,86.32,"Música. . Entonces, quisiera comenzar por con..."
1,Lorena Aldana - Using Python to listen to your...,https://www.youtube.com/watch?v=YrCSME1jT6U,PyCon-co audios/audios/Lorena Aldana - Using P...,200,PyCon Colombia,2021-09-09,[],UCjor6U0ZF5zGAYLJJt9gr0A,2,86.32,92.60,"palabra listen, escuchar y acá de pronto si t..."
2,Lorena Aldana - Using Python to listen to your...,https://www.youtube.com/watch?v=YrCSME1jT6U,PyCon-co audios/audios/Lorena Aldana - Using P...,200,PyCon Colombia,2021-09-09,[],UCjor6U0ZF5zGAYLJJt9gr0A,3,92.60,97.40,se los recomiendo vamos a estar escuchando al...
3,Lorena Aldana - Using Python to listen to your...,https://www.youtube.com/watch?v=YrCSME1jT6U,PyCon-co audios/audios/Lorena Aldana - Using P...,200,PyCon Colombia,2021-09-09,[],UCjor6U0ZF5zGAYLJJt9gr0A,4,97.40,103.08,tienen audígonos no se preocupen no hay probl...
4,Lorena Aldana - Using Python to listen to your...,https://www.youtube.com/watch?v=YrCSME1jT6U,PyCon-co audios/audios/Lorena Aldana - Using P...,200,PyCon Colombia,2021-09-09,[],UCjor6U0ZF5zGAYLJJt9gr0A,5,103.08,106.36,"sería una buena idea, si les, digamos, a la m..."


Para una busqueda lo mejor es combinar los textos para que no se pierda el contexto.

**IMPORTANTE: Para busqueda contextual hay que combinar los textos**

In [35]:
new_data = []

window = 6
stride = 3

for i in tqdm(range(0, len(df_transcripts), stride)):
  i_end = min(len(df_transcripts)-1, i + window)
  if df_transcripts.iloc[i]['title'] != df_transcripts.iloc[i_end]['title']:
    continue
  text = ' '.join(df_transcripts[i: i_end]['text'])
  new_data.append({
      "start" : df_transcripts.iloc[i]["start"],
      "end" : df_transcripts.iloc[i_end]["end"],
      "title" : df_transcripts.iloc[i]["title"],
      "views" : df_transcripts.iloc[i]["views"],
      "publish_date" : df_transcripts.iloc[i]["publish_date"],
      "keywords" : df_transcripts.iloc[i]["keywords"],
      "text": text,
      "id": df_transcripts.iloc[i]["position"],
      "url": df_transcripts.iloc[i]["url"],
  })

100%|██████████| 1958/1958 [00:02<00:00, 821.55it/s]


In [38]:
df_overlap = pd.DataFrame(new_data)

## Embeddings

In [39]:
from sentence_transformers import SentenceTransformer, util
import torch

In [41]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [43]:
sentences = ["Esto es un ejemplo", "Este es otro ejemplo"]
embeddings = model.encode (sentences)

In [46]:
len(embeddings[1])

384

In [47]:
embeddings = model.encode(df_overlap['text'], batch_size=64)

In [49]:
df_overlap["embeddings"] = embeddings.tolist()

In [50]:
df_overlap.sample(2)

,start,end,title,views,publish_date,keywords,text,id,url,embeddings
1806,1505.22,1545.18,David Delgado Ruiz - Story of an junior engine...,110,2019-05-24,[],"create a key and then put that zero, put a on...",343,https://www.youtube.com/watch?v=c8y2yS_p8wM,"[-0.05237895995378494, 0.07013235986232758, -0..."
1206,1701.76,1763.96,Navid Shelkhol - FBJE is a framework used at F...,152,2018-12-26,[],going to get notified somehow that there must...,254,https://www.youtube.com/watch?v=5rgZQw0F0w0,"[-0.07436706870794296, -0.07518796622753143, -..."


In [62]:
query =  model.encode(["Que es Machine Learning"])

In [63]:
df_overlap['similarity'] = df_overlap["embeddings"].apply(lambda x: util.cos_sim(x, query[0]))

In [64]:
df_overlap.sort_values('similarity', ascending=False).head(10)

,start,end,title,views,publish_date,keywords,text,id,url,embeddings,similarity
992,788.48,808.16,Sebastián Arango - Enhancing Data Privacy thro...,208,2020-03-06,[],"posean. Entonces, ¿qué es realmente el apren...",138,https://www.youtube.com/watch?v=mEPw36JE_8w,"[0.017229102551937103, -0.04375387355685234, -...",[[tensor(0.6721)]]
292,89.04,109.32,Rodolfo Edelmann & Carlos de la Torre - Conect...,503,2020-04-20,[],"Ahora en Mercado Libre estamos, como hizo Rud...",5,https://www.youtube.com/watch?v=N3czkxo2JJw,"[-0.012716985307633877, 0.01819167099893093, -...",[[tensor(0.6704)]]
1863,61.08,178.00,Rafael Carrascosa - Software engineering for m...,503,2021-08-05,[],El que se llama la. El que se llama la. El ...,35,https://www.youtube.com/watch?v=OVe1GzubDP8,"[-0.05274017155170441, -0.019284287467598915, ...",[[tensor(0.6415)]]
1876,714.40,779.00,Rafael Carrascosa - Software engineering for m...,503,2021-08-05,[],"estas y otras, en Mercado Libre tenemos proce...",74,https://www.youtube.com/watch?v=OVe1GzubDP8,"[-0.02382172830402851, 0.037150848656892776, -...",[[tensor(0.6105)]]
1864,64.08,205.98,Rafael Carrascosa - Software engineering for m...,503,2021-08-05,[],El que se llama la. El que se llama la. El qu...,38,https://www.youtube.com/watch?v=OVe1GzubDP8,"[-0.057171739637851715, -0.005716548301279545,...",[[tensor(0.6085)]]
1872,335.00,617.00,Rafael Carrascosa - Software engineering for m...,503,2021-08-05,[],Para planificar y estimar un proyecto que tie...,62,https://www.youtube.com/watch?v=OVe1GzubDP8,"[-0.04295634850859642, 0.01834726333618164, 0....",[[tensor(0.5897)]]
964,326.20,367.00,Sebastián Arango - Enhancing Data Privacy thro...,208,2020-03-06,[],haga posible identificar al usuario esté tota...,54,https://www.youtube.com/watch?v=mEPw36JE_8w,"[-0.03910747542977333, -0.0779929831624031, -0...",[[tensor(0.5723)]]
993,800.20,817.00,Sebastián Arango - Enhancing Data Privacy thro...,208,2020-03-06,[],"nuevo, Machine Learning. De hecho, utiliza l...",141,https://www.youtube.com/watch?v=mEPw36JE_8w,"[0.055914122611284256, -0.0326554998755455, -0...",[[tensor(0.5699)]]
1873,397.00,643.52,Rafael Carrascosa - Software engineering for m...,503,2021-08-05,[],El primero es la herramienta de factibilidad ...,65,https://www.youtube.com/watch?v=OVe1GzubDP8,"[-0.01800951361656189, 0.038617558777332306, -...",[[tensor(0.5664)]]
1886,253.56,304.12,Interview Rodolfo Edelmann & Carlos de la Torr...,274,2020-04-21,[],lo que es el Machine Learning en la organizac...,18,https://www.youtube.com/watch?v=IvKQkp2CYqs,"[0.04165314510464668, -0.005298386327922344, -...",[[tensor(0.5547)]]


In [65]:
import chromadb

client = chromadb.Client()
collection = client.create_collection("pycon") # Se le puede dar la ruta de un storage

In [69]:
df_overlap["id_database"] = df_overlap.apply(lambda x: str(hash(x["title"])) + "-" + str(x["id"]), axis=1)

In [70]:
collection.add(
    ids=df_overlap["id_database"].tolist(),
    embeddings=df_overlap["embeddings"].tolist(),
    metadatas= df_overlap[["start", "end", "text", "views", "publish_date", "url", "title"]].to_dict("records")
)

In [72]:
content = collection.query(
    query_texts = ["data visualization", "inteligencia artificial"], # By Default use sentence transformers
    n_results = 5
    )


# chromadb: https://docs.trychroma.com/embeddings

In [76]:
content['metadatas'][1]

[{'start': 89.04,
  'end': 109.32,
  'text': ' Ahora en Mercado Libre estamos, como hizo Rudy,  armando la infraestructura para ejecutar Machine Learning en  producción y a escala.  Bueno, queremos entonces contarles un poquito de qué va  a ser la charla.  Primero que todos, para entender el motivo de la charla,',
  'views': 503,
  'publish_date': '2020-04-20',
  'url': 'https://www.youtube.com/watch?v=N3czkxo2JJw',
  'title': 'Rodolfo Edelmann & Carlos de la Torre - Conectando microservicios con Python'},
 {'start': 2504.84,
  'end': 2599.3,
  'text': " their job is to observe and question, right? Like science is mostly about theory and asking why and how can we do this? Computer scientists especially, like right now, there's computer scientists around the world working on some incredible problems that will affect you in about 15 to 20 years. Like they're doing amazing research into type systems, into algorithms, into P  equals NP, into proofs, and all sorts of other things I don't ev